In [3]:
capture_data()

Patient Name: Reeshad Arian
Capture Heart Sound? (press Enter)
recording started
recording stopped
Capture Breath Sound? (press Enter)
recording started
recording stopped
Capture Speech? (press Enter)
recording started
recording stopped


In [8]:
import webbrowser
filename = 'database/Reeshad Arian/index.html'
webbrowser.open('file://' + os.path.realpath(filename))

True

In [9]:
import pyaudio
import webbrowser
import speech_recognition as sr
import wave
import matplotlib.pyplot as plt
import numpy as np
import  scipy.signal as signal
from collections import Counter
r = sr.Recognizer()
from IPython.display import Audio
import parselmouth
from numpy.fft import rfft
from numpy import argmax, mean, diff, log, nonzero
from scipy.signal import blackmanharris, correlate
from time import time
import parselmouth
from parselmouth.praat import call
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import os
from IPython.display import display, Markdown
import pickle

def capture_sound(sound_type, patient_name='', time=10, rate=44100):
    patient_name = 'database/' + patient_name + '/'
    if not os.path.exists(patient_name):
        os.makedirs(patient_name)
    RECORD_SECONDS = time
    if sound_type == "speech":
        WAVE_OUTPUT_FILENAME = patient_name+"speechFile.wav"
        index = 2
    elif sound_type == "heart":
        WAVE_OUTPUT_FILENAME = patient_name+ "heartFile.wav"
        index = 1
    else:
        WAVE_OUTPUT_FILENAME = patient_name+"breathFile.wav"
        index = 1
    audio = pyaudio.PyAudio()

    stream = audio.open(format=pyaudio.paInt16, channels=2,
                    rate=rate, input=True,input_device_index = index,
                    frames_per_buffer=1024)
    print ("recording started")
    Recordframes = []

    for i in range(0, int(rate / 1024 * time)):
        data = stream.read(1024)
        Recordframes.append(data)

    print ("recording stopped")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(2)
    waveFile.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    waveFile.setframerate(rate)
    waveFile.writeframes(b''.join(Recordframes))
    waveFile.close()
    file = open(patient_name+"transcriptFile.txt", "w")
    if sound_type == 'speech':
        with sr.WavFile(WAVE_OUTPUT_FILENAME) as source:              # use "test.wav" as the audio source
            audio = r.record(source)                        # extract audio data from the file
            try:
                 file.write(r.recognize_google(audio))   # recognize speech using Google Speech Recognition
            except:                                 # speech is unintelligible
                print("Could not understand audio")
        file.close()

def make_plots(filename, signal_type,  file_path):
    # Read file to get buffer                                                                                               
    ifile = wave.open(filename)
    samples = ifile.getnframes()
    audio = ifile.readframes(samples)
    
    # Convert buffer to float32 using NumPy                                                                                 
    audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
    audio_as_np_float32 = audio_as_np_int16.astype(np.float32)
    # Normalise float32 array so that values are between -1.0 and +1.0                                                      
    max_int16 = 2**15
    audio_normalised = audio_as_np_float32 / max_int16
    
    t = np.arange(0, 5*len(audio_normalised)/440320, 5/440320)
    plt.plot(t[:len(audio_normalised)], audio_normalised)
    plt.savefig(file_path+signal_type+"Plot.png")
    plt.close()
    s, f, _, i= plt.specgram(audio_normalised, Fs = 44100)
    plt.savefig(file_path+signal_type+"Spectogram.png")
    plt.close()
    Audio(filename)

def measurePitch(file_name, f0min = 75, f0max =500, unit = 'Hertz'):
    voiceID = parselmouth.Sound(file_name)
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

def EE(sig):
    X = np.fft.fft(sig)
    s = np.abs(sig)**2
    P = s/np.sum(s)+1e-40
    h = - np.sum(P*np.log2(P))
    return h

def zero_crossing_rate(sig):
    sig1 = np.append(0, sig)[:-1]
    return np.sum(np.abs(np.sign(sig)-np.sign(sig1)))/(2*len(sig))

def short_time_energy(sig, N):
    w = np.ones(N)/(2*N)
    return signal.convolve(sig**2, w**2)

def spectra_rolloff(sig, f_s, kappa=0.85):
    _, _, X = signal.spectrogram(sig, f_s)
    norm = X.sum(axis=0, keepdims=True)
    norm[norm == 0] = 1
    X = np.cumsum(X, axis=0) / norm
    vsr = np.argmax(X >= kappa, axis=0)
    # convert from index to Hz
    vsr = vsr / (X.shape[0] - 1)  * f_s / 2
    return (vsr)


def display_data(signal_type,  patient_name):
    res = ''
    file_path = 'database/' + patient_name + '/'
    filename = file_path + signal_type + 'File.wav'
    if signal_type == 'heart':
        title = 'Heart Data'
    elif signal_type == 'speech':
        title = 'Speech Data'
    else:
        title = 'Breathing Data'
    ifile = wave.open(filename)
    samples = ifile.getnframes()
    audio = ifile.readframes(samples)
    audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
    audio_as_np_float32 = audio_as_np_int16.astype(np.float32)
    max_int16 = 2**15
    audio_normalised = audio_as_np_float32 / max_int16
    yy = audio_normalised
    make_plots(filename, signal_type,  file_path)
    if signal_type == 'speech':
        res += 'Transcript: ' + open('database/'+patient_name+'/transcriptFile.txt').read() +'<br>'
    meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = measurePitch(filename)
    res += 'Mean F0: ' + str(meanF0) + '<br>'
    res += 'Stdev F0: ' + str(stdevF0) + '<br>'
    res += 'HNR: ' + str(hnr) + '<br>'
    res += 'Energy Entropy: ' + str(EE(yy)) + '<br>'
    res += 'Zero Crossing Rate: ' + str(zero_crossing_rate(yy)) + '<br>'
    res += 'Local Jitter: ' + str(localJitter) + '<br>'
    res += 'Local Absolute Jitter: ' + str(localabsoluteJitter) + '<br>'
    res += 'Rap Jitter: ' + str(rapJitter) + '<br>'
    res += 'ppq5 Jitter: ' + str(ppq5Jitter) + '<br>'
    res += 'ddp Jitter: ' + str(ddpJitter) + '<br>'
    res += 'Local Shimmer: ' + str(localShimmer) + '<br>'
    res += 'Local Shimmer dB: ' + str(localdbShimmer) + '<br>'
    res += 'apq3 Shimmer: ' + str(apq3Shimmer) + '<br>'
    res += 'aqpq5 Shimmer: ' + str(aqpq5Shimmer) + '<br>'
    res += 'apq11 Shimmer: ' + str(apq11Shimmer) + '<br>'
    res += 'dda Shimmer: ' + str(ddaShimmer) + '<br>'
    plt.plot(short_time_energy(yy, 2000))
    plt.savefig(file_path+signal_type+'shortEnergy.png')
    plt.close()
    return res

def capture_data(): 
    patient_name = input('Patient Name: ') 
    input('Capture Heart Sound? (press Enter)')
    capture_sound('heart', patient_name=patient_name, time=5)
    
    input('Capture Breath Sound? (press Enter)')
    capture_sound('breath', patient_name=patient_name, time=5)
    
    input('Capture Speech? (press Enter)')
    capture_sound('speech', patient_name=patient_name)
    
    heartStats = display_data('heart', patient_name)
    breathStats = display_data('breath', patient_name)
    speechStats = display_data('speech', patient_name)
    s = """window.onload = function(){
        document.getElementById('patientNameTitle').innerHTML = patientName + ' Report';
        document.getElementById('reportTitle').innerHTML = patientName + ' Report';
        document.getElementById('heartStats').innerHTML = heartStats;
        document.getElementById('breathStats').innerHTML = breathStats;
        document.getElementById('speechStats').innerHTML = speechStats;
    }"""
    contents = "var patientName = '" + patient_name + "';\n var heartStats = '" + heartStats + "';\n"
    contents += "var breathStats = '" + breathStats + "';\n"
    contents += "var speechStats = '" + speechStats + "';\n" + s
    outfile = open('database/'+patient_name+'/control.js', 'w')
    outfile.write(contents);
    outfile.close()
    s = """<!DOCTYPE html>
    <head><title id='patientNameTitle'></title></head>
    <body>
        <h1 id='reportTitle'></h1>
        <h2>Heart Sound Report</h2>
        <img src="./heartPlot.png" alt="Heart data plot" id="heartPlot">
        <img src="./heartSpectogram.png" alt="Heart Spectogram" id="heartSpectogram">
        <img src="./heartshortEnergy.png" alt="Heart Short Energy" id="heartShortEnergy">
        <audio controls>
            <source src="./heartFile.wav" type="audio/wav" id="heartAudio">
            Your browser does not support the audio element.
        </audio>
        <p id='heartStats'></p>
        <h2>Breath Sound Report</h2>
        <img src="./breathPlot.png" alt="Breath data plot" id="breathPlot">
        <img src="./breathSpectogram.png" alt="Breath Spectogram" id="breathSpectogram">
        <img src="./breathshortEnergy.png" alt="Breath Short Energy" id="breathShortEnergy">
        <audio controls>
            <source src="./breathFile.wav" type="audio/wav" id="breathAudio">
            Your browser does not support the audio element.
        </audio>
        <p id='breathStats'></p>
        <h2>Speech Sound Report</h2>
        <img src="./speechPlot.png" alt="Speech data plot" id="speechPlot">
        <img src="./speechSpectogram.png" alt="Speech Spectogram" id="speechSpectogram">
        <img src="./speechshortEnergy.png" alt="Speech Short Energy" id="speechShortEnergy">
        <audio controls>
            <source src="./speechFile.wav" type="audio/wav" id="speechAudio">
            Your browser does not support the audio element.
        </audio>
        <p id='speechStats'></p>
        <script src="control.js"></script>
    </body>"""
    outfile = open('database/'+patient_name+'/index.html', 'w')
    outfile.write(s)
    outfile.close()
    webbrowser.open('file://' + os.path.realpath('database/'+patient_name+'/index.html'))
    

In [5]:
B = np.array([[1, 0, 0, 1, 0], [1, 1, 0, 0, 1], [0, 1, 1, 0, 0], [0, 0, 1, 1, 1]])
B@B.T

array([[2, 1, 0, 1],
       [1, 3, 1, 1],
       [0, 1, 2, 1],
       [1, 1, 1, 3]])